In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np
import math as ma
import sys
np.seterr(divide='ignore', invalid='ignore')
iris = datasets.load_iris()


In [3]:
df = pd.DataFrame(iris.data)                                #creating dataframe for iris data
bf = pd.DataFrame(iris.target)                               #creating dataframe for iris target
df.columns = iris.feature_names                             #Assigning feature names to columns


In [4]:
def entropy(y):                                                         #To calculate entropy
    total_elements=len(y)                           
    no_of_setosa,no_of_versicolor,no_of_virginica = count(y)            #count of different individual classes
    total=no_of_setosa+no_of_versicolor+no_of_virginica                 #total count of individulal classes
    p1=no_of_setosa/total                                               
    p2=no_of_versicolor/total
    p3=no_of_virginica/total
    return_value=0
    if p1!=0:
        return_value+=p1*ma.log(p1,2)
    if p2!=0:
        return_value+=p2*ma.log(p2,2)
    if p3!=0:
        return_value+=p3*ma.log(p3,2) 
    if not return_value:
        return return_value
    return_value=-return_value                                          #entropy of current given y output data
    return return_value
    

In [5]:
def count(y):                                                           #to calculate count of setosa,versicolar and virginica
    setosa = np.array(y[:])
    count_setosa = (setosa==0).sum()
    
    versicolar = np.array(y[:])
    count_versi = (versicolar==1).sum()
    
    count_virgi = len(y) - (count_setosa + count_versi)
    return count_setosa,count_versi,count_virgi
  

In [6]:
def split(y1,y2,y):                                                     #to calculate the split info
    p1 = len(y1)/len(y)
    p2 = len(y2)/len(y)
    result = 0
    if p1!=0:
        result += p1*ma.log(p1,2)
    if p2!=0:
        result += p2*ma.log(p2,2)
    return -result

In [7]:
def feature_split(x,y,sf):                              #to calculate the best split value at which the gain ratio is maximum
    xe = np.array(x[sf])                                #creating a numpy array of values of x's sf column
    #xe.sort                                             
    temp_gain = -1                                      #a variable to max gain
    split_value = 0                                     #a variable to store best split value
    for i in range(1,len(x)):
        splitt = (xe[i-1] + xe[i])/2                    #calculating mid of two values of xe
       
        x_split1 = x[x[sf]>splitt]                      #splitting data of x where x[sf] is greater than splitt
        x_split2 = x[x[sf]<=splitt]                     #splitting data of x where x[sf] is less than or equal to splitt
        y_split1 = y[x[sf]>splitt]                      #splitting data of y where x[sf] is greater than splitt
        y_split2 = y[x[sf]<=splitt]                     #splitting data of y where x[sf] is less than or equal to splitt
        
        final_entropy = 0                               #variable for final entropy
        initial_entropy = entropy(y)                    #variable for initial entropy

        entropy1 = entropy(y_split1)                    #entropy of first split
        entropy2 = entropy(y_split2)                    #entropy of second split   
        final_entropy += entropy1 * len(y_split1)/len(y)                  
        final_entropy += entropy2 * len(y_split2)/len(y)                  #calculating final entropy
        info_gain = initial_entropy - final_entropy                       #calculating information gain
        split_info = split(y_split1,y_split2,y)                           #calculating split info
        gain_r = gain(info_gain,split_info)                               #calculating gain ratio
        if temp_gain < gain_r:                                            #to get maximum gain ratio and best split value
            temp_gain = gain_r
            split_value = splitt
            
    
    return temp_gain,split_value                                          #returning best gain ratio and split value of sf feature
        

In [8]:
def gain(info_gain,split_info):                                           #to calculate gain ratio
    
    gain_ratio = info_gain/split_info
    return gain_ratio
   

In [9]:
def dt(x,y,features,lvl):
    featuresLeft = features.copy()
    no_of_features_left=len(features)                                     
    total_elements=len(x)
    no_of_setosa,no_of_versicolor,no_of_virginica = count(y)     
 
    print('Level',lvl)                                                    #to tell at which depth the tree is currently
    print('Count of setosa = ',no_of_setosa)
    print('Count of versicolor = ',no_of_versicolor)
    print('Count of virginica =',no_of_virginica)
    print('Current Entropy is =',entropy(y))
    
    if no_of_setosa==total_elements or no_of_versicolor==total_elements or no_of_virginica==total_elements or no_of_features_left==0:
        print('Reached leaf node')
        print()
        return 
    else:
        maxg = -1                                                         #variable to hold maximum gain ratio value
        split_value = 0                                                   #variable to hold best split value
        indexValue = 0
        for i in features:
            temp_gain,temp_split_value = feature_split(x,y,i)             #getting maximum gain and best split value for feature i
            if maxg < temp_gain:                                          #to get the maximum gain among different features
                maxg = temp_gain                                            
                split_value = temp_split_value                            #to get the best split value among different features
                split_feature = i                                         #best feature to split on
                used = indexValue
            indexValue += 1
        featuresLeft.pop(used)
        print('Splitting on feature',split_feature,'with gain ratio',maxg)
        print()
        
        x1 = x[x[split_feature]>split_value]                               #splitting data of x where x[split_feature] is greater than split_value
        y1 = y[x[split_feature]>split_value]                               #splitting data of y where x[split_feature] is greater than split_value
        x2 = x[x[split_feature]<=split_value]                              #splitting data of x where x[split_feature] is less than or equal to split_value
        y2 = y[x[split_feature]<=split_value]                              #splitting data of y where x[split_feature] is less than or equal to split_value
        
        dt(x1,y1,featuresLeft,lvl+1)                                           #using recursion to get the values
        dt(x2,y2,featuresLeft,lvl+1)
    

In [10]:
dt(df,bf,list(df.columns),0)                                                     #Main function call

Level 0
Count of setosa =  50
Count of versicolor =  50
Count of virginica = 50
Current Entropy is = 1.584962500721156
Splitting on feature petal width (cm) with gain ratio 0.9999999999999999

Level 1
Count of setosa =  0
Count of versicolor =  50
Count of virginica = 50
Current Entropy is = 1.0
Splitting on feature petal length (cm) with gain ratio 0.6621582046482519

Level 2
Count of setosa =  0
Count of versicolor =  6
Count of virginica = 49
Current Entropy is = 0.4971677614160753
Splitting on feature sepal length (cm) with gain ratio 0.05463893158092842

Level 3
Count of setosa =  0
Count of versicolor =  0
Count of virginica = 12
Current Entropy is = 0.0
Reached leaf node

Level 3
Count of setosa =  0
Count of versicolor =  6
Count of virginica = 37
Current Entropy is = 0.5830194167347008
Splitting on feature sepal width (cm) with gain ratio 0.0519480903515746

Level 4
Count of setosa =  0
Count of versicolor =  0
Count of virginica = 5
Current Entropy is = 0.0
Reached leaf node
